# Empresa de Aviação

Usando Gurobi para otimizar

In [1]:
#!/usr/bin/env  julia
using JuMP
using Gurobi

m = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

Declarando as variáveis

In [2]:
# Conjunto de aviões
A = ["A1", "A2", "A3"]

# Custos dos aviões
C = Dict(
    "A1" => 5.1,
    "A2" => 3.6,
    "A3" => 6.8
)

# Receita Teórica
R = Dict(
    "A1" => 330,
    "A2" => 300,
    "A3" => 420
)

# Pilotos aptos
P = Dict(
    "A1" => 30,
    "A2" => 20,
    "A3" => 10
)

# Relação da capacidade de manuteção do hangar
M = Dict(
    "A1" => 1,
    "A2" => 0.75,
    "A3" => 1.66666666666667
)

capacidadeMax = 40  #capacidade máxima de manutenção do hangar
verba = 220

220

Váriáveis do problema que queremos otimizar, são a quantidade de cada modelo de avião que a empresa deve comprar para otimizar seu lucro 

In [3]:
@variable(m, x[A] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, ["A1", "A2", "A3"]
And data, a 3-element Array{VariableRef,1}:
 x[A1]
 x[A2]
 x[A3]

Função objetivo do problema

In [4]:
# Implementação da função objetivo
@objective(m, Max, sum(x[i]*(R[i] - C[i]) for i in A))

324.9 x[A1] + 296.4 x[A2] + 413.2 x[A3]

Adicionando restrições do problema

In [5]:
@constraints(m, 
    begin
        sum(x[i]*M[i] for i in A) <= capacidadeMax
        sum(x[i]*C[i] for i in A) <= verba
        x["A3"]*2 <= P["A3"]
        x["A1"]*2 + x["A3"]*2 <= P["A1"] + P["A3"]
        x["A2"]*2 + x["A3"]*2 <= P["A2"] + P["A3"]
        sum(x[i]*2 for i in A) <= sum(P[i] for i in A)
    end)

println(m)

Max 324.9 x[A1] + 296.4 x[A2] + 413.2 x[A3]
Subject to
 x[A1] ≥ 0.0
 x[A2] ≥ 0.0
 x[A3] ≥ 0.0
 x[A1] + 0.75 x[A2] + 1.66666666666667 x[A3] ≤ 40.0
 5.1 x[A1] + 3.6 x[A2] + 6.8 x[A3] ≤ 220.0
 2 x[A3] ≤ 10.0
 2 x[A1] + 2 x[A3] ≤ 40.0
 2 x[A2] + 2 x[A3] ≤ 30.0
 2 x[A1] + 2 x[A2] + 2 x[A3] ≤ 60.0



Otimizando o problema e exibindo os resultados

In [6]:
start = time()
optimize!(m)
println("Tempo: $(time()-start)s")

println("Valor ótimo: $(map(i-> JuMP.value(x[i]),A))")
println("Função ótima: ", JuMP.objective_value(m))

Academic license - for non-commercial use only
Optimize a model with 6 rows, 3 columns and 14 nonzeros
Coefficient statistics:
  Matrix range     [8e-01, 7e+00]
  Objective range  [3e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2714742e+04   1.000800e+01   0.000000e+00      0s
       2    9.9035000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds
Optimal objective  9.903500000e+03
Tempo: 5.2021708488464355s
Valor ótimo: [15.0, 10.0, 5.0]
Função ótima: 9903.5
